In [1]:
import sys
sys.path.append("..")

import torch
from torch.optim import AdamW
from transformers import T5Tokenizer
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration

from datasets import load_dataset
from transformers import get_linear_schedule_with_warmup

from scripts.global_vars import (
    DEVICE, 
    MAX_TURNS,
    BATCH_SIZE, 
    MODEL_NAME,
    USE_TRAINED_MODEL,
    MAX_LENGTH_ENCODER_ACTION, 
    MAX_LENGTH_DECODER_ACTION
)

from scripts.utils import find_zero_percentage
from scripts.pytorch.training import train_model
from scripts.pytorch.inference import inference_model
from scripts.preprocessing.action import ActionDataset, get_evaluation_score

In [2]:
dataset = load_dataset("multi_woz_v22", trust_remote_code=True)

train_data = dataset['train']
val_data = dataset['validation']

In [3]:
tokenizer = T5Tokenizer.from_pretrained(
    legacy=True,
    pretrained_model_name_or_path=MODEL_NAME
)

train_action_dataset = ActionDataset(
    data=dataset['train'],
    tokenizer=tokenizer,
    max_turns=MAX_TURNS,
    max_output_len=MAX_LENGTH_DECODER_ACTION,
    max_input_len=MAX_LENGTH_ENCODER_ACTION
)

valid_action_dataset = ActionDataset(
    data=dataset['validation'],
    tokenizer=tokenizer,
    max_turns=MAX_TURNS,
    max_output_len=MAX_LENGTH_DECODER_ACTION,
    max_input_len=MAX_LENGTH_ENCODER_ACTION
)

train_loader_action = DataLoader(train_action_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader_action = DataLoader(valid_action_dataset, batch_size=BATCH_SIZE)

batch = next(iter(train_loader_action))
print("Inputs IDs shape:", batch['encoder_input_ids'].shape)
print("Action IDs shape:", batch['decoder_input_ids'].shape)

Processing dialogues: 100%|██████████| 1000/1000 [00:00<00:00, 2098.74it/s]


Inputs IDs shape: torch.Size([128, 64])
Action IDs shape: torch.Size([128, 64])


In [4]:
train_encoder_zero = find_zero_percentage(train_loader_action, "encoder_input_ids", MAX_LENGTH_ENCODER_ACTION)
train_decoder_zero = find_zero_percentage(train_loader_action, "decoder_input_ids", MAX_LENGTH_DECODER_ACTION)
valid_encoder_zero = find_zero_percentage(valid_loader_action, "encoder_input_ids", MAX_LENGTH_ENCODER_ACTION)
valid_decoder_zero = find_zero_percentage(valid_loader_action, "decoder_input_ids", MAX_LENGTH_DECODER_ACTION)

In [5]:
print(
    "Train Encoder Zero Percentage:", train_encoder_zero * 100,
    "\nTrain Decoder Zero Percentage:", train_decoder_zero * 100,
    "\nValid Encoder Zero Percentage:", valid_encoder_zero * 100,
    "\nValid Decoder Zero Percentage:", valid_decoder_zero * 100
)

Train Encoder Zero Percentage: 33.90093445777893 
Train Decoder Zero Percentage: 58.621543645858765 
Valid Encoder Zero Percentage: 32.69616961479187 
Valid Decoder Zero Percentage: 58.01507234573364


In [ ]:
num_epochs = 5
num_training_steps = len(train_loader_action) * num_epochs
num_warmup_steps = num_training_steps // 10

action_model = T5ForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path=MODEL_NAME
).to(DEVICE)

optimizer = AdamW(
    action_model.parameters(),
    lr=1e-3,
    eps=1e-8
)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [ ]:
if not USE_TRAINED_MODEL:
    action_model = train_model(
        action_model,
        optimizer,
        scheduler,
        train_loader_action,
        valid_loader_action,
        num_epochs=num_epochs,
        device=DEVICE,
        save="../../models/multixoz_action_model.pth"
    )
    
else:
    action_model.load_state_dict(torch.load("../../models/multixoz_action_model.pth"))


Epoch 1/5
--------------------------------------------------


Training:   0%|          | 0/444 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1004.00 MiB. GPU 0 has a total capacity of 15.70 GiB of which 300.62 MiB is free. Process 270471 has 6.50 GiB memory in use. Including non-PyTorch memory, this process has 8.89 GiB memory in use. Of the allocated memory 8.63 GiB is allocated by PyTorch, and 21.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
generated_action_train = inference_model(
    action_model, 
    tokenizer, 
    train_action_dataset.inputs, 
    MAX_LENGTH_ENCODER_ACTION, 
    MAX_LENGTH_DECODER_ACTION, 
    DEVICE,
    batch_size=1024
) 

generated_action_valid = inference_model(
    action_model, 
    tokenizer, 
    valid_action_dataset.inputs, 
    MAX_LENGTH_ENCODER_ACTION, 
    MAX_LENGTH_DECODER_ACTION, 
    DEVICE,
    batch_size=1024
) 

Inference: 100%|██████████| 8/8 [00:14<00:00,  1.80s/it]


In [ ]:
bleu_score_train = get_evaluation_score(generated_action_train, train_action_dataset.actions)
bleu_score_valid = get_evaluation_score(generated_action_valid, valid_action_dataset.actions)

In [ ]:
print("BLEU Score Train:", bleu_score_train["bleu_score"])
print("SER Score Train:", bleu_score_train["ser_score"])
print("BLEU Score Valid:", bleu_score_valid["bleu_score"])
print("SER Score Valid:", bleu_score_valid["ser_score"])

BLEU Score Train: 0.6406678874822183
SER Score Train: 0.7302708508960378
BLEU Score Valid: 0.6376035060365202
SER Score Valid: 0.7286841698045081


In [ ]:
index = 1
inputs = train_action_dataset.inputs[index]

generated_action = inference_model(
    action_model,
    tokenizer,
    inputs,
    MAX_LENGTH_ENCODER_ACTION,
    MAX_LENGTH_DECODER_ACTION,
    DEVICE
)

print("User Inputs:", inputs)
print("Generated Action:", generated_action)
print("True Action:", train_action_dataset.actions[index])

Inference: 100%|██████████| 1/1 [00:00<00:00,  8.18it/s]

User Inputs: USER: Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation? SYS: There is an Afrian place named Bedouin in the centre. How does that sound?
Generated Action: Restaurant-Inform(area=centre, name=Afrian place, name=Bedouin) | Restaurant-Recommend
True Action: Restaurant-Inform(area=centre, food=Afrian, name=Bedouin)


In [ ]:
generated_action_train

['Restaurant-Inform(choice=several) | Restaurant-Select(food=African, food=British,',
 'Restaurant-Inform(area=centre, name=Afrian place, name=Bedouin) | Restaurant-Recommend',
 "Hotel-Recommend(area=center of town, name=Bedouin's Hotel, phone=01223367660,",
 'Booking-Request(bookday=?)',
 'Booking-Book(ref=FRGZWQL2) | general-reqmore(none=none)',
 'general-bye(none=none)',
 'Hotel-Inform(choice=4, choice=two, pricerange=cheap, type=guesthouses, type=hotels',
 'Hotel-Request(pricerange=?)',
 'Booking-Inform(none=none) | Hotel-Recommend(name=aylesbray lodge guest house)',
 'Booking-NoBook(none=none) | Booking-Request(bookday=?, bookstay=?)',
 'Booking-Book(ref=84ESP6F5',
 'general-bye(none=none)',
 'Booking-Inform(none=none) | Hotel-Inform(address=Sleeperz Hotel, address=C',
 'Booking-Request(bookstay=?)',
 'Booking-Book(ref=CYJDAR7R) | general-reqmore(none=none)',
 'Restaurant-Inform(area=west, choice=9, pricerange=expensive) | Restaurant-Request(food=?)',
 'Booking-Inform(none=none) |